In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/t5-small-standard-bahasa-cased')

In [4]:
from glob import glob

checkpoints = sorted(glob('finetune-t5-tiny-standard-bahasa-cased/checkpoint-*'))
checkpoints

['finetune-t5-tiny-standard-bahasa-cased/checkpoint-280000',
 'finetune-t5-tiny-standard-bahasa-cased/checkpoint-290000',
 'finetune-t5-tiny-standard-bahasa-cased/checkpoint-300000']

In [14]:
from unidecode import unidecode
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [15]:
p_wikipedia = """
Najib razak telah dipilih untuk Parlimen Malaysia pada tahun 1976,
pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang.
Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang,
sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan.
Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran.
Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi,
sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008.
Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013,
walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""
q_wikipedia = ['bilakah najib dipilih untuk parlimen malaysia', 
               'Apakah jawatan yang pernah dipegang oleh Najib Razak?']

In [16]:
text = cleaning(p_wikipedia)
input_ids = []
for q in q_wikipedia:
    s = f'konteks: {text} soalan: {q}'
    # print(s)
    # s = f'teks: {text} entiti: {t}'
    input_ids.append({'input_ids': tokenizer.encode(s, return_tensors='pt')[0]})

padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 256, num_beams=5, 
    early_stopping=True)
b = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for no, q in enumerate(q_wikipedia):
    print(q, ':', b[no])

bilakah najib dipilih untuk parlimen malaysia : 1976
Apakah jawatan yang pernah dipegang oleh Najib Razak? : Kabinet


In [17]:
p_news = """
Bekas perdana menteri Najib Razak mempersoalkan tindakan polis yang menurutnya tidak serta-merta mengeluarkan kenyataan berhubung dakwaan Adun Perikatan Nasional (PN) "merancang" insiden rogol.
Sedangkan, kata ahli parlimen Pekan itu, polis pantas mengeluarkan kenyataan apabila dia dilapor terlupa mengimbas MySejahtera sebelum masuk restoran.
"Berita Najib lupa scan MySejahtera tular, kenyataan polis terus keluar. Berita Dr Mahathir Mohamad lupa scan, kenyataan, polis serta-merta keluar.
"Sebab itu saya pelik kenapa pihak polis belum sempat keluar apa-apa kenyataan berhubung kes seorang gadis membuat laporan polis untuk dakwa Adun PN rancang insiden rogolnya," katanya di Facebook hari ini.
Najib merujuk dakwaan seorang wanita yang mendakwa dirogol kenalan kepada Adun Gombak Setia, Hilman Idham.
Wanita itu mendakwa ahli politik dari Bersatu berkenaan merancang insiden yang berlaku pada 5 Dis lalu.
Menurut laporan polis pada 8 Mei, mangsa mendakwa kejadian itu berlaku di sebuah hotel di Selangor, yang pada masa itu berada di bawah perintah kawalan pergerakan bersyarat (PKPB).
"""

q_news = ['siapakah yang mempersoalkan tindakan polis', 'siapakah Adun Gombak Setia',
         'siapakah ahli perlimen Pekan?', 'who is adun gombak setia',
         'bilakah tarikh laporan polis tentang kejadian di sebuah hotel di Selangor']

In [18]:
text = cleaning(p_news)
input_ids = []
for q in q_news:
    s = f'konteks: {text} soalan: {q}'
    # print(s)
    # s = f'teks: {text} entiti: {t}'
    input_ids.append({'input_ids': tokenizer.encode(s, return_tensors='pt')[0]})

padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 256)
b = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for no, q in enumerate(q_news):
    print(q, ':', b[no])

siapakah yang mempersoalkan tindakan polis : Bekas perdana menteri Najib Razak
siapakah Adun Gombak Setia : Hilman Idham
siapakah ahli perlimen Pekan? : parlimen Pekan itu
who is adun gombak setia : Hilman Idham
bilakah tarikh laporan polis tentang kejadian di sebuah hotel di Selangor : 8 Mei


In [19]:
model.push_to_hub('finetune-extractive-qa-t5-tiny-standard-bahasa-cased', organization='mesolitica')

Upload file pytorch_model.bin:   0%|          | 32.0k/133M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mesolitica/finetune-extractive-qa-t5-tiny-standard-bahasa-cased
   d01ba07..da4caf7  main -> main



'https://huggingface.co/mesolitica/finetune-extractive-qa-t5-tiny-standard-bahasa-cased/commit/da4caf7c08931a6731ca7e9d666611d9ce6cebbd'

In [20]:
tokenizer.push_to_hub('finetune-extractive-qa-t5-tiny-standard-bahasa-cased', organization='mesolitica')

Upload file spiece.model:   4%|4         | 32.0k/784k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mesolitica/finetune-extractive-qa-t5-tiny-standard-bahasa-cased
   da4caf7..0d0d252  main -> main



'https://huggingface.co/mesolitica/finetune-extractive-qa-t5-tiny-standard-bahasa-cased/commit/0d0d252f5661def960218e61fa956d51d15cc37a'

In [13]:
import re
import collections
import string

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [14]:
import json

with open('ms-dev-2.0.json') as fopen:
    data = json.load(fopen)

In [15]:
from tqdm import tqdm

srcs, answers = [], []
for i in tqdm(range(len(data['data']))):
    for p in data['data'][i]['paragraphs']:
        text = p['context']
        if len(text.split()) > 500:
            continue
        
        for q in p['qas']:
            qs = q['question']
            is_impossible = q.get('is_impossible', False)
            if is_impossible:
                a = 'tiada jawapan'
            else:
                a = q['answers'][0]['text']

            if not len(a):
                a = 'tiada jawapan'

            src = f'konteks: {text} soalan: {qs}'
            input_ids = {'input_ids': tokenizer.encode(src, return_tensors='pt')}
            srcs.append(input_ids)
            answers.append(a)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:05<00:00,  6.45it/s]


In [16]:
exact, f1 = [], []

for i in tqdm(range(len(srcs))):
    a = answers[i]
    outputs = model.generate(**srcs[i], max_length = 256)
    b = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    f1.append(compute_f1(a, b))
    exact.append(compute_exact(a, b))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 11849/11849 [08:17<00:00, 23.83it/s]


In [17]:
import numpy as np

np.mean(exact), np.mean(f1)

(0.4269558612541143, 0.5113033923646412)

In [18]:
import json

with open('dev-v2.0.json') as fopen:
    data = json.load(fopen)

In [19]:
srcs, answers = [], []
for i in tqdm(range(len(data['data']))):
    for p in data['data'][i]['paragraphs']:
        text = p['context']
        if len(text.split()) > 500:
            continue
        
        for q in p['qas']:
            qs = q['question']
            is_impossible = q.get('is_impossible', False)
            if is_impossible:
                a = 'tiada jawapan'
            else:
                a = q['answers'][0]['text']

            if not len(a):
                a = 'tiada jawapan'

            src = f'konteks: {text} soalan: {qs}'
            input_ids = {'input_ids': tokenizer.encode(src, return_tensors='pt')}
            srcs.append(input_ids)
            answers.append(a)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:05<00:00,  6.43it/s]


In [20]:
exact, f1 = [], []

for i in tqdm(range(len(srcs))):
    a = answers[i]
    outputs = model.generate(**srcs[i], max_length = 256)
    b = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    f1.append(compute_f1(a, b))
    exact.append(compute_exact(a, b))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 11840/11840 [08:39<00:00, 22.78it/s]


In [21]:
np.mean(exact), np.mean(f1)

(0.46925675675675677, 0.5410633845304322)